In [1]:
import numpy as np
import pandas as pd
from gurobipy import *

# For progress bar
# Instead of writing...
# > for i in some_list:
# You can write....
# > for i in tqdm(some_list):
# And you will get a very nice progress bar. This is very nice for 
# for-loops that run a long time, like running an optimization model for 
# many timesteps :)
from tqdm.auto import tqdm, trange  

# Plotting libaries - Feel free to use any others of your choice
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.style as style
style.use('fivethirtyeight')

# Enable high resolution plots
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

# HINT  
I recommend that you read through this notebook once *before* you start coding - you can do it all on your own but if you adhere to the structure that we hint at in this notebook, it might be easier for you

## Reading data  
This data is pretty clean but there are some surprises... I recommend that you re-index the dataframe with a [datetime-index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) since they have some very convienient attributes and methods which you will need (unless you want to put more effort into this).  
So for example the entries "01-01-2019", "00 - 01" should be changed so that this row has a datetime index "2019-01-01 00:00:00"

In [2]:
def read_ElspotPrices(filename):
    df = pd.read_html(filename, decimal=',', thousands='.', header = 2, match = 'Elspot Prices in EUR/MWh')
    df = df[0]
    df.rename( columns={'Unnamed: 0':'Dates'}, inplace=True )
    df['Hours'] = pd.Series(df['Hours'], dtype="string")
    df['Hours'] = df['Hours'].str.slice(0,3)
    df['Hours'] = df['Hours'] + '- 00'
    df['Hours'] = pd.Series(df['Hours'], dtype="object")
    df['Datetime'] = pd.to_datetime(df['Dates'] + ' ' + df['Hours'], format='%d-%m-%Y %H - %M')
    df = df.set_index('Datetime')
    df = df.drop(['Dates','Hours'], axis=1)
    df.dropna(axis=1, how='all', inplace=True)
    df.columns = df.columns.str.replace('.1', '')
    df.rename(columns={'S': 'SE1', 'D': 'DK1'}, inplace=True)
    return df # Returning a pandas dataframe

In [3]:
path = 'https://www.nordpoolgroup.com/4a05c7/globalassets/marketdata-excel-files/elspot-prices_2019_hourly_eur.xls'
df = read_ElspotPrices(path) # You can rename variables
df.columns

/Users/hhk/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


Index(['SYS', 'SE1', 'SE2', 'SE3', 'SE4', 'FI', 'DK1', 'DK2', 'Oslo',
       'Kr.sand', 'Bergen', 'Molde', 'Tr.heim', 'TromsÃ¸', 'EE', 'LV', 'LT',
       'AT', 'BE', 'DE-LU', 'FR', 'NL'],
      dtype='object')

#### Daylight savings time  
The data from Elspot contains one hour twice and one hour is missing, thanks to daylight savings time. You have to convert the data so that the timezone is without daylight savings (so no hours exist twice or are missing).
You should also convert the data to UTC+0. The Nordpool headquarters are in norway (CET with summer time, so during the summer they have UTC+2 and during winter UTC+1). 
Below an example of the day light savings switch and how it should look like after you converted the time.  
Hint: You can use pandas functions for this, it can be done in three lines of code 

In [4]:
# Find the day light savings switch with duplicated datetime (clocks turned back for 1 hour)
# You can remove this cell if you want
timediffs = pd.Series(df.index).diff() # Calculate time difference between indices
daylight_idx = timediffs[timediffs == pd.Timedelta(0)].idxmax() #  Find row with no time-difference 
df[daylight_idx-2:daylight_idx+2] # Show the rows surrounding this row

,SYS,SE1,SE2,SE3,SE4,FI,DK1,DK2,Oslo,Kr.sand,...,Tr.heim,TromsÃ¸,EE,LV,LT,AT,BE,DE-LU,FR,NL
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-10-27 01:00:00,33.84,31.68,31.68,31.68,31.68,31.68,27.11,27.11,37.12,37.12,...,31.68,31.68,31.68,31.68,31.68,32.01,26.30,-34.57,27.30,27.11
2019-10-27 02:00:00,33.43,31.46,31.46,31.46,31.46,31.46,9.04,9.04,37.11,37.11,...,31.46,31.46,31.46,31.46,31.46,31.07,14.25,-29.97,21.13,25.00
2019-10-27 02:00:00,33.18,31.49,31.49,31.49,31.49,31.49,0.05,0.05,37.11,37.11,...,31.49,31.49,31.49,31.49,31.49,31.05,25.81,-9.97,11.58,25.70
2019-10-27 03:00:00,33.05,31.12,31.12,31.12,31.12,31.12,0.12,0.12,37.08,37.08,...,31.36,31.36,31.12,31.12,31.12,31.04,22.76,0.12,14.03,22.86


In [5]:
def removeDaylightSavings(df):
    df.dropna(axis=0, how='all', inplace=True)
    df.index = df.index.tz_localize('Europe/Berlin', ambiguous = 'infer')
    df.index = df.index.tz_convert(None)
    return df

In [6]:
df = removeDaylightSavings(df)

In [7]:
# Show results
df[daylight_idx-2:daylight_idx+2]

,SYS,SE1,SE2,SE3,SE4,FI,DK1,DK2,Oslo,Kr.sand,...,Tr.heim,TromsÃ¸,EE,LV,LT,AT,BE,DE-LU,FR,NL
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-10-27 00:00:00,33.43,31.46,31.46,31.46,31.46,31.46,9.04,9.04,37.11,37.11,...,31.46,31.46,31.46,31.46,31.46,31.07,14.25,-29.97,21.13,25.00
2019-10-27 01:00:00,33.18,31.49,31.49,31.49,31.49,31.49,0.05,0.05,37.11,37.11,...,31.49,31.49,31.49,31.49,31.49,31.05,25.81,-9.97,11.58,25.70
2019-10-27 02:00:00,33.05,31.12,31.12,31.12,31.12,31.12,0.12,0.12,37.08,37.08,...,31.36,31.36,31.12,31.12,31.12,31.04,22.76,0.12,14.03,22.86
2019-10-27 03:00:00,33.32,31.66,31.66,31.66,31.66,31.66,5.50,5.50,37.00,37.00,...,31.66,31.66,31.66,31.66,31.66,31.03,16.41,5.50,12.90,16.47


This is how your data should be formated that you use as input for the "prediction generator":  
* pd.Series() containing the prices for your market area
* index should be of type datetime -> This will make it much easier to create your plots later

In [8]:
example = df["SE1"]
example

Datetime
2018-12-31 23:00:00    28.32
2019-01-01 00:00:00    10.07
2019-01-01 01:00:00    10.03
2019-01-01 02:00:00     4.56
2019-01-01 03:00:00     4.83
                       ...  
2019-12-31 18:00:00    32.37
2019-12-31 19:00:00    31.23
2019-12-31 20:00:00    31.33
2019-12-31 21:00:00    31.46
2019-12-31 22:00:00    30.83
Name: SE1, Length: 8760, dtype: float64

In [9]:
df.index

DatetimeIndex(['2018-12-31 23:00:00', '2019-01-01 00:00:00',
               '2019-01-01 01:00:00', '2019-01-01 02:00:00',
               '2019-01-01 03:00:00', '2019-01-01 04:00:00',
               '2019-01-01 05:00:00', '2019-01-01 06:00:00',
               '2019-01-01 07:00:00', '2019-01-01 08:00:00',
               ...
               '2019-12-31 13:00:00', '2019-12-31 14:00:00',
               '2019-12-31 15:00:00', '2019-12-31 16:00:00',
               '2019-12-31 17:00:00', '2019-12-31 18:00:00',
               '2019-12-31 19:00:00', '2019-12-31 20:00:00',
               '2019-12-31 21:00:00', '2019-12-31 22:00:00'],
              dtype='datetime64[ns]', name='Datetime', length=8760, freq=None)

## "Predicting" future prices  
Since this is not a data science course, we will compare two very simple "prediction" methods:  
1) We perfectly know future prices  
2) Assume price will be the same as last weeks price at the same time  
  
If you want, you can also implement more elaborate prediction methods - you may be able to beat the results from 2) but you should not be able to beat 1)   
In case you implement another predictor, you should obviously only use the prices up to t-1 to predict prices for t to t + n :)

In [10]:
# You do not have to touch this code
class PredictionGenerator():
    def __init__(self, timeseries, method="perfect"):
        self.setMethod(method)
        self.df = timeseries

    # If you use more elaborate prediction methods, you might have to fit a model to the data
    # Implementing another prediction method will not better your grade, so ignore this if you want.
    def fit(self, end_date):
        pass

    # "Predict" prices for some date range defined by a start date and a number of hours
    def predict(self, start_date, nhours):
        dates = pd.date_range(
            start=start_date,
            periods=nhours,
            freq="H")

        if self.method == "Perfect":
            if dates.max() not in self.df.index: # check if prices known
                raise Exception(("You can not forecast the *future* perfectly "
                                 "out of sample! Some of the dates requested "
                                 "are not in the given dataset."))
            result = self.df.loc[dates]  # the actual observed prices
        elif self.method == "SameHourLastWeek":
            min_date = (dates.min() + pd.DateOffset(hours=-7*24))
            max_date = (dates.max() + pd.DateOffset(hours=-7*24))
            if (min_date not in self.df.index) or (max_date not in self.df.index):
                raise Exception(("For some of the dates you tried to predict, "
                                 "there is no data from the week before available."))
            # Last weeks prices
            result = self.df.loc[dates + pd.DateOffset(hours=-7*24)]
            result.index = dates
            
        return result

    
    # Set method if known, otherwise raise exception
    def setMethod(self, method):
        methods = ["Perfect", "SameHourLastWeek"]
        if method not in methods:
            raise Exception(f"Method passed({method}) is not in ["
                            + ", ".join(methods) + "]")
        self.method = method

In [11]:
# Example how to use:
# Create predictor
prediction_method = "SameHourLastWeek" # methods = ["Perfect", "SameHourLastWeek"]
prices = df["SE1"] # All prices that we have for the market area of interest as a Series
predictor = PredictionGenerator(prices, method=prediction_method)
# "Predict" prices for some time frame
t_start = pd.Timestamp("2019-10-22 00:00:00")
predicted_prices = predictor.predict(t_start, 10) # returns a series of "predicted" prices
                                                  # for the 10 hours starting on t_start


## The gurobi model  

In [12]:
def solve(prices,
          Pmax,
          Emax,
          E0,
          print_result=False):
    
    model = Model("Aluminum DSM")
    model.modelSense = GRB.MINIMIZE
    
    # If you test the model for one timestep, you probably want the usual gurobi
    # output, but if you run this model a lot (for every time-step...) this will
    # spam your output window, so i recommend to set print_result=False
    if not print_result:
        model.setParam('OutputFlag', False)

    # Times = []
    # for i in prices.index:
    #     Times.append(i.value)

    Times = prices.index

    # Power deviation from nominal
    delta_P = {}
    for t in Times:
        delta_P[t] = model.addVar(
            name="delta_P_%s" % (t), 
            lb = -Pmax,
            ub = Pmax, 
            vtype=GRB.CONTINUOUS
        )

    # Energy deviation from nominal
    delta_E = {}
    for t in Times:
        delta_E[t] = model.addVar(
            name="delta_E_%s" % (t),
            lb = -Emax,
            ub = Emax, 
            vtype=GRB.CONTINUOUS
        )

    # # Maximum power difference
    # for t in Times:
    #     model.addConstr(-Pmax <= delta_P[t])
    #     model.addConstr(delta_P[t] <= Pmax)

    # # Maximum energy difference
    # for t in Times:
    #     model.addConstr(-Emax <= delta_E[t])
    #     model.addConstr(delta_E[t] <= Emax)

    # Energy balance
    for num_t in range(len(Times)):
        if num_t != 0:
            model.addConstr(delta_E[Times[num_t]] == 
                            delta_E[Times[num_t - 1]] + delta_P[Times[num_t]] * 1)

    # Energy at t = tau is given
    model.addConstr(delta_E[Times[0]] == 
                    E0 + delta_P[Times[0]] * 1)

    # No energy difference at last time step
    model.addConstr(delta_E[Times[-1]] == 0)

    # Obj function
    model.setObjective(
        quicksum( prices[t] * delta_P[t] * 1 for t in Times)
    )

    model.update()
    model.optimize()

    result_P = {}
    result_E = {}
    for t in Times:
        result_P[t] = float(delta_P[t].x)
        result_E[t] = float(delta_E[t].x)
    
    # In tutor you have to return the model, but in the notebook
    # you can also return something different (like P, E for all timesteps)
    return result_P, result_E

# Running your scenario  
The following cell is just to show you how to use tqdm for a progress bar, you can delete that cell and write the code to run the scenario how you want :)

In [13]:
def runScenario(historic_prices,             # Historic electricty prices
                E0, Emax, Pmax,              # Plant parameters
                prediction_method,           # Prediction settings
                t_start, t_end, t_horizon):  # Simulation times

    # You probably have to prepare some stuff here
    datetime = pd.date_range(t_start, t_end, freq="H")
    results = pd.DataFrame({'Datetime': datetime})
    results['Pt'] = np.nan
    results['Et'] = np.nan
    results['current_price'] = historic_prices[datetime].to_list()
    results['predicted_price'] = np.nan
    results['cummulative_earnings'] = np.nan
    results = results.set_index('Datetime')
    
    for t in tqdm(pd.date_range(t_start, t_end, freq="H")):
        
        # Prepare price data for model

        # check t + t_horizon > t_end
        if t + pd.Timedelta(t_horizon, 'H') > t_end:
            diff_hours = (t + pd.Timedelta(t_horizon, 'H') - t_end).total_seconds()/ (60 * 60)
            t_hours = t_horizon - diff_hours + 1
        else:
            t_hours = t_horizon

        # "Predict" prices
        predictor = PredictionGenerator(historic_prices, method=prediction_method)
        predicted_prices = predictor.predict(t, t_hours)

        # Run model & save results for current timestep
        P_iter, E_iter = solve(predicted_prices, Pmax, Emax, E0, print_result=False)
        results.loc[t,'Pt'] = P_iter[t]
        results.loc[t,'Et'] = E_iter[t]
        results.loc[t,'predicted_price'] = predicted_prices[0]

        earnings = 0
        for iter_t in pd.date_range(t_start, t, freq="H"):
            earnings = earnings - results.loc[iter_t, 'predicted_price'] * results.loc[iter_t, 'Pt'] * 1

        results.loc[t, 'cummulative_earnings'] = earnings

        # "Update world state":
        # - Set E0 based on optimization results for current time step
        E0 = E0 + P_iter[t] * 1
     
        # Write current results to console
        tqdm.write("%s: P = %s, E = %s, earnings = %s" % (t, P_iter[t], E0, earnings))
        
    return results # I recommend storing your results in a dataframe as well, for easier plotting

In [14]:
# Get data
market_area = "SE1"
historic_prices = df[market_area]

# Plant parameters
E0 = 0                                   # Energy difference at t = t_start
Pmax = 100                               # Maximum power deviation in MW
Emax = 48 * Pmax                         # Maximum energy deviation in MWh

# Simulation parameters
t_start = pd.Timestamp("2019-10-22 00:00:00") # Times are in UTC+0
t_end = pd.Timestamp("2019-12-22 00:00:00")  # Simulate two months of operation
t_horizon = 7*24                             # Time horizon for optimization (in h)

In [15]:
# Run your scenario :D
results_perfect = runScenario(historic_prices, E0, Emax, Pmax, 'Perfect', t_start, t_end, t_horizon)
results_lastWeek = runScenario(historic_prices, E0, Emax, Pmax, 'SameHourLastWeek', t_start, t_end, t_horizon)

  0%|          | 0/1465 [00:00<?, ?it/s]

Academic license - for non-commercial use only - expires 2021-08-02
Using license file /Users/hhk/gurobi.lic
2019-10-22 00:00:00: P = 100.0, E = 100.0, earnings = -3381.0
2019-10-22 01:00:00: P = 100.0, E = 200.0, earnings = -6565.0
2019-10-22 02:00:00: P = 100.0, E = 300.0, earnings = -9934.0
2019-10-22 03:00:00: P = 100.0, E = 400.0, earnings = -13392.0
2019-10-22 04:00:00: P = 100.0, E = 500.0, earnings = -16963.0
2019-10-22 05:00:00: P = -100.0, E = 400.0, earnings = -13285.0
2019-10-22 06:00:00: P = -100.0, E = 300.0, earnings = -9564.0
2019-10-22 07:00:00: P = -100.0, E = 200.0, earnings = -5891.0
2019-10-22 08:00:00: P = -100.0, E = 100.0, earnings = -2248.0
2019-10-22 09:00:00: P = 100.0, E = 200.0, earnings = -5826.0
2019-10-22 10:00:00: P = 100.0, E = 300.0, earnings = -9378.0
2019-10-22 11:00:00: P = 100.0, E = 400.0, earnings = -12866.0
2019-10-22 12:00:00: P = 100.0, E = 500.0, earnings = -16332.0
2019-10-22 13:00:00: P = 100.0, E = 600.0, earnings = -19803.0
2019-10-22 14

2019-10-27 09:00:00: P = 100.0, E = 4200.0, earnings = -94874.0
2019-10-27 10:00:00: P = 100.0, E = 4300.0, earnings = -98540.0
2019-10-27 11:00:00: P = 100.0, E = 4400.0, earnings = -102160.0
2019-10-27 12:00:00: P = 100.0, E = 4500.0, earnings = -105764.0
2019-10-27 13:00:00: P = 100.0, E = 4600.0, earnings = -109401.0
2019-10-27 14:00:00: P = 100.0, E = 4700.0, earnings = -113115.0
2019-10-27 15:00:00: P = -100.0, E = 4600.0, earnings = -109266.0
2019-10-27 16:00:00: P = -100.0, E = 4500.0, earnings = -105214.0
2019-10-27 17:00:00: P = -100.0, E = 4400.0, earnings = -101108.0
2019-10-27 18:00:00: P = -100.0, E = 4300.0, earnings = -97075.0
2019-10-27 19:00:00: P = -100.0, E = 4200.0, earnings = -93223.0
2019-10-27 20:00:00: P = -100.0, E = 4100.0, earnings = -89465.0
2019-10-27 21:00:00: P = 100.0, E = 4200.0, earnings = -93120.0
2019-10-27 22:00:00: P = 100.0, E = 4300.0, earnings = -96623.0
2019-10-27 23:00:00: P = 100.0, E = 4400.0, earnings = -100171.0
2019-10-28 00:00:00: P = 1

2019-11-01 19:00:00: P = 100.0, E = 500.0, earnings = 71303.0
2019-11-01 20:00:00: P = 100.0, E = 600.0, earnings = 67677.0
2019-11-01 21:00:00: P = 100.0, E = 700.0, earnings = 64273.0
2019-11-01 22:00:00: P = 100.0, E = 800.0, earnings = 61195.0
2019-11-01 23:00:00: P = 100.0, E = 900.0, earnings = 58045.0
2019-11-02 00:00:00: P = 100.0, E = 1000.0, earnings = 55038.0
2019-11-02 01:00:00: P = 100.0, E = 1100.0, earnings = 52237.0
2019-11-02 02:00:00: P = 100.0, E = 1200.0, earnings = 49537.0
2019-11-02 03:00:00: P = 100.0, E = 1300.0, earnings = 46731.0
2019-11-02 04:00:00: P = 100.0, E = 1400.0, earnings = 43721.0
2019-11-02 05:00:00: P = 100.0, E = 1500.0, earnings = 40278.0
2019-11-02 06:00:00: P = 100.0, E = 1600.0, earnings = 36708.0
2019-11-02 07:00:00: P = 100.0, E = 1700.0, earnings = 33049.0
2019-11-02 08:00:00: P = 100.0, E = 1800.0, earnings = 29254.0
2019-11-02 09:00:00: P = 100.0, E = 1900.0, earnings = 25411.0
2019-11-02 10:00:00: P = 100.0, E = 2000.0, earnings = 21594

2019-11-07 05:00:00: P = -100.0, E = 2300.0, earnings = 68170.0
2019-11-07 06:00:00: P = -100.0, E = 2200.0, earnings = 73560.0
2019-11-07 07:00:00: P = -100.0, E = 2100.0, earnings = 79425.0
2019-11-07 08:00:00: P = -100.0, E = 2000.0, earnings = 84712.0
2019-11-07 09:00:00: P = -100.0, E = 1900.0, earnings = 89661.0
2019-11-07 10:00:00: P = -100.0, E = 1800.0, earnings = 94450.0
2019-11-07 11:00:00: P = -100.0, E = 1700.0, earnings = 99165.0
2019-11-07 12:00:00: P = -100.0, E = 1600.0, earnings = 103848.0
2019-11-07 13:00:00: P = -100.0, E = 1500.0, earnings = 108521.0
2019-11-07 14:00:00: P = -100.0, E = 1400.0, earnings = 113263.0
2019-11-07 15:00:00: P = -100.0, E = 1300.0, earnings = 118251.0
2019-11-07 16:00:00: P = -100.0, E = 1200.0, earnings = 123494.0
2019-11-07 17:00:00: P = -100.0, E = 1100.0, earnings = 128313.0
2019-11-07 18:00:00: P = -100.0, E = 1000.0, earnings = 132985.0
2019-11-07 19:00:00: P = -100.0, E = 900.0, earnings = 137483.0
2019-11-07 20:00:00: P = -100.0, 

...

2019-11-28 21:00:00: P = 100.0, E = -2600.0, earnings = 437419.0
2019-11-28 22:00:00: P = 100.0, E = -2500.0, earnings = 433754.0
2019-11-28 23:00:00: P = 100.0, E = -2400.0, earnings = 430309.0
2019-11-29 00:00:00: P = 100.0, E = -2300.0, earnings = 426920.0
2019-11-29 01:00:00: P = 100.0, E = -2200.0, earnings = 423500.0
2019-11-29 02:00:00: P = 100.0, E = -2100.0, earnings = 420050.0
2019-11-29 03:00:00: P = 100.0, E = -2000.0, earnings = 416533.0
2019-11-29 04:00:00: P = 100.0, E = -1900.0, earnings = 412786.0
2019-11-29 05:00:00: P = 100.0, E = -1800.0, earnings = 408763.0
2019-11-29 06:00:00: P = -100.0, E = -1900.0, earnings = 413050.0
2019-11-29 07:00:00: P = -100.0, E = -2000.0, earnings = 417366.0
2019-11-29 08:00:00: P = -100.0, E = -2100.0, earnings = 421609.0
2019-11-29 09:00:00: P = -100.0, E = -2200.0, earnings = 425843.0
2019-11-29 10:00:00: P = -100.0, E = -2300.0, earnings = 430077.0
2019-11-29 11:00:00: P = -100.0, E = -2400.0, earnings = 434322.0
2019-11-29 12:00:00

2019-12-04 07:00:00: P = -100.0, E = -1200.0, earnings = 425374.0
2019-12-04 08:00:00: P = -100.0, E = -1300.0, earnings = 429918.0
2019-12-04 09:00:00: P = -100.0, E = -1400.0, earnings = 434312.0
2019-12-04 10:00:00: P = -100.0, E = -1500.0, earnings = 438705.0
2019-12-04 11:00:00: P = -100.0, E = -1600.0, earnings = 443117.0
2019-12-04 12:00:00: P = -100.0, E = -1700.0, earnings = 447504.0
2019-12-04 13:00:00: P = -100.0, E = -1800.0, earnings = 451825.0
2019-12-04 14:00:00: P = -100.0, E = -1900.0, earnings = 456166.0
2019-12-04 15:00:00: P = -100.0, E = -2000.0, earnings = 460537.0
2019-12-04 16:00:00: P = -100.0, E = -2100.0, earnings = 464981.0
2019-12-04 17:00:00: P = -100.0, E = -2200.0, earnings = 469264.0
2019-12-04 18:00:00: P = -100.0, E = -2300.0, earnings = 473444.0
2019-12-04 19:00:00: P = -100.0, E = -2400.0, earnings = 477562.0
2019-12-04 20:00:00: P = -100.0, E = -2500.0, earnings = 481633.0
2019-12-04 21:00:00: P = 0.0, E = -2500.0, earnings = 481633.0
2019-12-04 22

2019-12-09 18:00:00: P = -100.0, E = -3500.0, earnings = 547965.0
2019-12-09 19:00:00: P = -100.0, E = -3600.0, earnings = 552141.0
2019-12-09 20:00:00: P = -100.0, E = -3700.0, earnings = 556303.0
2019-12-09 21:00:00: P = -100.0, E = -3800.0, earnings = 560465.0
2019-12-09 22:00:00: P = -100.0, E = -3900.0, earnings = 564513.0
2019-12-09 23:00:00: P = 100.0, E = -3800.0, earnings = 560565.0
2019-12-10 00:00:00: P = 100.0, E = -3700.0, earnings = 556651.0
2019-12-10 01:00:00: P = 100.0, E = -3600.0, earnings = 552748.0
2019-12-10 02:00:00: P = 100.0, E = -3500.0, earnings = 548898.0
2019-12-10 03:00:00: P = 100.0, E = -3400.0, earnings = 544949.0
2019-12-10 04:00:00: P = -100.0, E = -3500.0, earnings = 549033.0
2019-12-10 05:00:00: P = -100.0, E = -3600.0, earnings = 553026.0
2019-12-10 06:00:00: P = -100.0, E = -3700.0, earnings = 557040.0
2019-12-10 07:00:00: P = -100.0, E = -3800.0, earnings = 561179.0
2019-12-10 08:00:00: P = -100.0, E = -3900.0, earnings = 565278.0
2019-12-10 09:0

2019-12-15 04:00:00: P = 100.0, E = -1200.0, earnings = 505348.0
2019-12-15 05:00:00: P = 100.0, E = -1100.0, earnings = 502500.0
2019-12-15 06:00:00: P = 100.0, E = -1000.0, earnings = 499272.0
2019-12-15 07:00:00: P = 100.0, E = -900.0, earnings = 495858.0
2019-12-15 08:00:00: P = 100.0, E = -800.0, earnings = 492331.0
2019-12-15 09:00:00: P = 100.0, E = -700.0, earnings = 488734.0
2019-12-15 10:00:00: P = 100.0, E = -600.0, earnings = 485096.0
2019-12-15 11:00:00: P = 100.0, E = -500.0, earnings = 481521.0
2019-12-15 12:00:00: P = 100.0, E = -400.0, earnings = 477929.0
2019-12-15 13:00:00: P = 100.0, E = -300.0, earnings = 474377.0
2019-12-15 14:00:00: P = 100.0, E = -200.0, earnings = 470825.0
2019-12-15 15:00:00: P = -100.0, E = -300.0, earnings = 474570.0
2019-12-15 16:00:00: P = -100.0, E = -400.0, earnings = 478334.0
2019-12-15 17:00:00: P = 100.0, E = -300.0, earnings = 474639.0
2019-12-15 18:00:00: P = 100.0, E = -200.0, earnings = 471055.0
2019-12-15 19:00:00: P = 100.0, E =

2019-12-20 14:00:00: P = -100.0, E = -2400.0, earnings = 611381.0
2019-12-20 15:00:00: P = -100.0, E = -2500.0, earnings = 615196.0
2019-12-20 16:00:00: P = -100.0, E = -2600.0, earnings = 618996.0
2019-12-20 17:00:00: P = -100.0, E = -2700.0, earnings = 622795.0
2019-12-20 18:00:00: P = -100.0, E = -2800.0, earnings = 626591.0
2019-12-20 19:00:00: P = -100.0, E = -2900.0, earnings = 630331.0
2019-12-20 20:00:00: P = 100.0, E = -2800.0, earnings = 626680.0
2019-12-20 21:00:00: P = 100.0, E = -2700.0, earnings = 623159.0
2019-12-20 22:00:00: P = 100.0, E = -2600.0, earnings = 619763.0
2019-12-20 23:00:00: P = 100.0, E = -2500.0, earnings = 616324.0
2019-12-21 00:00:00: P = 100.0, E = -2400.0, earnings = 613062.0
2019-12-21 01:00:00: P = 100.0, E = -2300.0, earnings = 609879.0
2019-12-21 02:00:00: P = 100.0, E = -2200.0, earnings = 606774.0
2019-12-21 03:00:00: P = 100.0, E = -2100.0, earnings = 603661.0
2019-12-21 04:00:00: P = 100.0, E = -2000.0, earnings = 600484.0
2019-12-21 05:00:00

Since running the model for many time-steps may take a long time, i recommend storing the results. Then when you start working on the visualization on a later day, you can just load the stored results instead of running your model again. One easy way to store python objects is using pickle, which is also integrated in pandas, see this example:

In [16]:
# If your results are a pandas dataframe, you can store them very easily as a pickle file
pd.to_pickle(results_perfect, "perfect.pkl") 
pd.to_pickle(results_lastWeek, "lastWeek.pkl") 

In [17]:
# And when you want to continue working later, you can reload them by running this cell
# You will only have to run the first cell (imports) and this one, and none of the others
results_perfect = pd.read_pickle("perfect.pkl")
results_lastWeek = pd.read_pickle("lastWeek.pkl")

## Plotting the results  
For these graphs, you should plot the results for both the "perfect prediction" and the "last week prediction". If you have implemented any other prediction method, you can also implement that.
For your results, please plot...
* ... P(t) for the first 100 hours
* ... E(t) for the first 100 hours
* ... cummulative earnings over the entire time  

Using just the "last week prediction", plot...
* ... the average P(t) and p(t) (power and price) for each hour of the day and each day of the week

To show how the actual plant operation changes as new data becomes available, plot (for the first week, using only "last week prediction")...
* ... actual E(t) vs E(t) as planned in first timestep


#### Plot P(t) for the first 100 hours

#### Plot E(t) for the first 100 hours

#### Plot earnings

#### Average Power and electricity price for each hour of the day and each day of the week  
Please note that the second y axis is inverted. This is not a data viz course, so sue me

#### Plot actual E(t) for the first week versus E(t) from first iteration of optimization